In [ ]:
from math import pi, cos, sin, sqrt
import sys
sys.path.append('/root/alok/repos/cv_research/alok')

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
from mpl_toolkits.mplot3d import Axes3D
import statsmodels.api as sm
from obb import OBB
from biomass_utils.points_of_interest import get_data, get_depth_cutoff, get_points_of_interest, distance_between_points

In [ ]:
np.median(np.array([1, 2, 3]))

In [ ]:
data_dir_base = '/root/data/blender_v3'
idx = 8

data = get_data(data_dir_base, idx)
point_cloud, mask, annotation, image = data['point_cloud'], data['mask'], data['annotation_data'], data['image']

# clean mask
i_values, j_values = np.where(mask > 0)
j_min, j_max = np.min(j_values), np.max(j_values)
for j in xrange(j_min, j_max+1):
    i_values = np.where(mask[:, j] > 0)[0]
    i_min, i_max = np.min(i_values), np.max(i_values)
    i_min_depth_value_mean = point_cloud[:,:,1][i_min-5:i_min+5, j].mean()
    i_max_depth_value_mean = point_cloud[:,:,1][i_max-5:i_max+5, j].mean()
    cutoff_depth_value = min(i_min_depth_value_mean, i_max_depth_value_mean)
    mask[:, j] = mask[:, j] * (point_cloud[:,:,1][:, j] < cutoff_depth_value)

fish_point_cloud = point_cloud[mask > 0]
depth_histogram = np.histogram(fish_point_cloud[:,1])
obb, eigen_vectors = OBB.build_from_points([(p[0], p[1], p[2]) for p in fish_point_cloud])
obb_points = np.array(obb.points)

In [ ]:
[(p[0], p[1], p[2]) for p in fish_point_cloud]

In [ ]:
length = np.linalg.norm(obb_points[0] - obb_points[1])
width = np.linalg.norm(obb_points[0] - obb_points[3])
height = np.linalg.norm(obb_points[0] - obb_points[5])
print(length, width, height)
print(annotation['length'])
print(annotation['width'])
print(annotation['height'])

In [ ]:
fig = plt.figure()
ax = Axes3D(fig)

# get x, y, and z lists
x_values = list(fish_point_cloud[:,0]) + list(obb_points[:,0])
y_values = list(fish_point_cloud[:,1]) + list(obb_points[:,1])
z_values = list(fish_point_cloud[:,2]) + list(obb_points[:,2])

ax.scatter(x_values, y_values, z_values)
# ax.view_init(30, 40)
for point_pair in [(0, 1), (1, 2), (2, 3), (3, 0), \
                   (4, 5), (5, 6), (6, 7), (7, 4), \
                   (0, 5), (1, 4), (2, 7), (3, 6)]:
    i, j = point_pair
    edge_x_values = [obb_points[i][0], obb_points[j][0]]
    edge_y_values = [obb_points[i][1], obb_points[j][1]]
    edge_z_values = [obb_points[i][2], obb_points[j][2]]
    ax.plot(edge_x_values, edge_y_values, edge_z_values)
# ax.plot([0, 30*eigen_vectors[0][0]], [0, 30*eigen_vectors[0][1]], [0, 30*eigen_vectors[0][2]])
# ax.plot([0, 30*eigen_vectors[1][0]], [0, 30*eigen_vectors[1][1]], [0, 30*eigen_vectors[1][2]])
# ax.plot([0, 30*eigen_vectors[2][0]], [0, 30*eigen_vectors[2][1]], [0, 30*eigen_vectors[2][2]])
plt.show()

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(data['mask'])

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(point_cloud[:,:,1])

In [ ]:
Image.open('{}/stereo_images/left_0.png'.format(data_dir_base))

In [ ]:
data_dir_base = '/root/data/blender_v3'
idx_max = 5000

analysis_df = pd.DataFrame()
for idx in range(idx_max):
    if idx == 1454:
        continue
    
    if idx % 100 == 0:
        print(idx)
    
    data = get_data(data_dir_base, idx)
    point_cloud, mask, annotation, image = data['point_cloud'], data['mask'], data['annotation_data'], data['image']

    # clean mask
    i_values, j_values = np.where(mask > 0)
    j_min, j_max = np.min(j_values), np.max(j_values)
    for j in xrange(j_min, j_max+1):
        i_values = np.where(mask[:, j] > 0)[0]
        i_min, i_max = np.min(i_values), np.max(i_values)
        i_min_depth_value_mean = point_cloud[:,:,1][i_min-5:i_min+5, j].mean()
        i_max_depth_value_mean = point_cloud[:,:,1][i_max-5:i_max+5, j].mean()
        cutoff_depth_value = min(i_min_depth_value_mean, i_max_depth_value_mean)
        mask[:, j] = mask[:, j] * (point_cloud[:,:,1][:, j] < cutoff_depth_value)

    # get obb
    fish_point_cloud = point_cloud[mask > 0]
    depth_histogram = np.histogram(fish_point_cloud[:,1])
    obb, eigen_vectors = OBB.build_from_points([(p[0], p[1], p[2]) for p in fish_point_cloud])
    obb_points = np.array(obb.points)
    
    # get dimensions    
    length = np.linalg.norm(obb_points[0] - obb_points[1])
    width = np.linalg.norm(obb_points[0] - obb_points[3])
    height = np.linalg.norm(obb_points[0] - obb_points[5])
    obb_volume = length * width * height
    
    row_to_append = {
        'ground_truth_length': annotation['length'],
        'ground_truth_width': annotation['width'],
        'ground_truth_height': annotation['height'],
        'ground_truth_volume': annotation['volume'],
        'predicted_length': max(length, width, height),
        'obb_volume': length * width * height
    }
    analysis_df = analysis_df.append(row_to_append, ignore_index=True)
    
    
    
    

In [ ]:
mask = analysis_df.obb_volume / analysis_df.ground_truth_volume < 6
plt.scatter(analysis_df[mask].ground_truth_volume, analysis_df[mask].obb_volume)
plt.show()

In [ ]:
features = ['obb_volume']
target = 'ground_truth_volume'
TRAIN_SIZE = 2500

mask = (analysis_df.obb_volume / analysis_df.ground_truth_volume < 6) & (analysis_df.index < TRAIN_SIZE)
y = analysis_df.ix[mask, target]
X = analysis_df.ix[mask, features]
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
print(model.summary())

A = np.vstack([analysis_df.ix[mask, features].values.T, np.ones(analysis_df[mask].shape[0])]).T
res = np.linalg.lstsq(A, analysis_df.ix[mask, target].values)
b, a = res[0]

In [ ]:
analysis_df['predicted_volume'] = model.params.const + model.params.obb_volume * analysis_df.obb_volume

In [ ]:
test_mask = (analysis_df.obb_volume / analysis_df.ground_truth_volume < 6) & (analysis_df.index >= TRAIN_SIZE)
(analysis_df.ix[test_mask, 'predicted_volume'].mean() - analysis_df.ix[test_mask, 'ground_truth_volume'].mean())/(analysis_df.ix[test_mask, 'ground_truth_volume'].mean())